# The low-level storage-only API

This guide walks through the low-level storage API `lamin.storage`.

The typical user will never need to use this API. The only functionality provided is saving and reading plain files without metadata.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbproject import Header

Header(filepath="storage.ipynb");  # just a temporary hack, here

In [ ]:
import lamindb as ln
import scanpy as sc
import pytest  # only needed to demonstrate exceptions

In [ ]:
adata = sc.datasets.pbmc68k_reduced()

## Local mode

Let's configure lamin to write files to a directory "data" in our home:

In [ ]:
!lamindb configure --storage "$HOME/data"

Let's create an instance of file:

In [ ]:
file = ln.storage.file.h5ad("test-file.h5ad")

And write some data:

In [ ]:
file.write(adata)

It's now visible in the data folder we just created:

In [ ]:
!ls $HOME/data

reading the file works as follows:

In [ ]:
adata = ln.storage.file.h5ad("test-file.h5ad").read()

As `h5ad` files have a 1:1 mapping with `AnnData` objects, we can type-check the user input.

If we try to write a DataFrame to an `.h5ad`, this will raise a `TypeError`.

In [ ]:
with pytest.raises(TypeError):
    file.write(adata.to_df())

## Cloud mode on S3

Let us now configure storage in the cloud, here, on AWS S3.

We need a cache directory for backing data from the cloud locally, and hence provide it to the configuration in addition to the S3 bucket.

In [ ]:
!lamindb configure \
    --storage 's3://lamin0' \
    --cache '$HOME/.lamin/cache'

Besides this configuration, nothing changes for the user. We can still use arbitrary file paths and file names, as if the data was local.

In [ ]:
file = ln.storage.file.h5ad("test-file.h5ad")

By checking the file path, you see that your file now lives in remote storage:

In [ ]:
file.path

And you can still use all the attributes that you're used from a `Path` object.

In [ ]:
file.path.parts

Saving works as before, too.

In [ ]:
file.write(adata)

As does reading.

In [ ]:
%%time

adata = ln.storage.file.h5ad("test-file.h5ad").read()

If the cache file disappears,

In [ ]:
!rm ~/.lamin/cache/lamin0/test-file.h5ad

the data in the cloud will be auto-downloaded:

In [ ]:
%%time

adata = ln.storage.file.h5ad("test-file.h5ad").read()

In [ ]:
!ls ~/.lamin/cache/lamin0/test-file.h5ad